In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-ad279118'


create qapp
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
loading module simple_controls
loading module save_resampled
loading module selection
loading module bsa_proba
loading module modelGraphs
loading module profilewindow
loading module ana_image_math
loading module paletteViewer
loading module foldsplit
loading module anacontrolmenu
loading module gradientpalette
loading module palettecontrols
loading module meshsplit
loading module volumepalettes
loading module gltf_io
loading module infowindow
loading module histogram
loading module statsplotwindow
loading module valuesplotwindow
all python modules loaded
Anatomist started.


In [2]:
from soma import aims
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
import umap
import os

### Region name

In [2]:
side = "R" # "R" or "L"
region = "CINGULATE" # "ORBITAL", "CINGULATE", "SC-sylv", "F.I.P."

NameError: name 'pd' is not defined

## Visualization

In [1]:
def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

def buckets_average(subject_id_list, dataset_name_list):
    dic_vol = {}
    dim = 0
    rep = 0
    if len(subject_id_list) == 0:
        return False
    while dim == 0 and rep < len(subject_id_list):
        if dataset_name_list[rep].lower() in ['ukb', 'ukbiobank', 'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset_name_list[rep].lower() in ['hcp']:
            dataset = 'hcp'
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"
        if os.path. isfile(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz'):
            sum_vol = aims.read(f'{mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz').astype(float)
            dim = sum_vol.shape
            sum_vol.fill(0)
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id_list[rep]}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id_list[0]}_cropped_skeleton.nii.gz')
        rep += 1

    for subject_id, dataset in zip(subject_id_list,dataset_name_list):
        if dataset.lower() in ['ukb', 'ukbiobank',  'projected_ukb']:
            dataset = 'UkBioBank'
        elif dataset.lower() == 'hcp':
            dataset = 'hcp'
            
        mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops"

        if os.path. isfile(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz'):
            vol = aims.read(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            # compare the dim with the first file dim

            if vol.np.shape != dim:
                raise ValueError(f"{subject_id_list[0]} and {subject_id} must have the same dim")

                
            # to have a binary 3D structure
            dic_vol[subject_id] = (vol.np > 0).astype(int)
            sum_vol.np[:] += (vol.np > 0).astype(int) 
        else: 
            print(f'FileNotFound {mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')
            #raise FileNotFoundError(f'{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz')

    sum_vol = sum_vol / len(subject_id_list)
    print(sum_vol, sum_vol.shape)
    return sum_vol

In [35]:
average_dic = {}
nb_columns = 10
block = a.createWindowsBlock(nb_columns)

In [ ]:
predict_proba = pd.DataFrame([0.05,0.06,0.02,0.01], index = ['sub-1521212','sub-1521211','sub-1421212','sub-1621212'])

In [36]:
step = 200
dic_packages = {}
for i in range(0,len(predict_proba),step):
    list_idx = (predict_proba.index[i:i+step].to_numpy())
    dic_packages[i//step] = [f'{idx}' for idx in list_idx] 
list_database = ['hcp' for i in range(step)]
n_pack = len(dic_packages)

In [37]:
#for i in ([k for k in range(10)]+[k for k in range(n_pack//2-5,n_pack//2+5)]+[k for k in range(n_pack-10,n_pack)]):
for i in range(0,n_pack, n_pack//30):
    sum_vol = buckets_average(dic_packages[i], list_database)
    average_dic[f'a_sum_vol{i}'] = a.toAObject(sum_vol)
    average_dic[f'a_sum_vol{i}'].setPalette(minVal=0, absoluteMode=True)
    #wsum = a.createWindow('Sagittal', block=block)
    #wsum.addObjects(a_sum_vol)
    average_dic[f'rvol{i}'] = a.fusionObjects(objects=[average_dic[f'a_sum_vol{i}']], method='VolumeRenderingFusionMethod')
    average_dic[f'rvol{i}'].releaseAppRef()
    # custom palette
    n = len(dic_packages[i])
    pal = a.createPalette('VR-palette')
    pal.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
    #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.5/n};0;1;1'
    build_gradient(pal)
    average_dic[f'rvol{i}'].setPalette('VR-palette', minVal=0.05, absoluteMode=True)
    pal2 = a.createPalette('slice-palette')
    pal2.header()['palette_gradients'] = '0;0.459574;0.497872;0.910638;1;1#0;0;0.52766;0.417021;1;1#0;0.7;1;0#0;0;0.0297872;0.00851064;0.72766;0.178723;0.957447;0.808511;1;1'
    #f'0;0.244444;0.5;1;1;1#0;0;0.535897;0.222222;1;1#0;0.7;1;0#0;0;{0.3/n};0;{0.7/n};1;1;1'
    build_gradient(pal2)
    average_dic[f'a_sum_vol{i}'].setPalette('slice-palette')
    # rvol.palette().fill()
    average_dic[f'wvr{i}'] = a.createWindow('3D', block=block)
    average_dic[f'wvr{i}'].addObjects(average_dic[f'rvol{i}'])

<soma.aims.Volume_DOUBLE object at 0x763a52fe8af0> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a52febac0> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a80f2e0> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a80ecb0> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a7a5e10> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a7a5630> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a7f0790> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a7f1bd0> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a80eef0> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a800790> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a801240> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a784b80> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a791d80> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a791090> (16, 37, 37, 1)
<soma.aims.Volume_DOUBLE object at 0x763a5a7ad000> (16, 37, 37

no position could be read at -1, 181
no position could be read at 92, 17
no position could be read at 72, 10
no position could be read at 134, 138
no position could be read at 116, 168
no position could be read at 40, 203


SetObjectPaletteCommand : warning: palette "CustomGradient" not found


no position could be read at 142, 173
no position could be read at 119, 206
